In [1]:
import pandas as pd
import talib
import numpy as np
import matplotlib.pyplot as plt
from importlib import reload
from datetime import datetime
import time
import sqlite3

In [2]:
import datafunc; reload(datafunc)
from datafunc import *

In [4]:
df = pd.read_csv('all_dailies.csv')

In [16]:
df.date.min()

'2019-04-08'

In [12]:
year = df[(df['date'] > '2024-01-01') & (df['date'] <= '2024-05-31')]

In [13]:
year.date.max()

'2024-05-31'

In [14]:
year.date.min()

'2024-01-02'

In [15]:
year.shape

(1101539, 9)

In [4]:
df['rank'] = get_ranks(df)

In [5]:
df['return'] = df['close'].groupby(df['ticker']).pct_change()
df['cumret'] = df.groupby('ticker')['return'].transform(lambda x: (1 + x).cumprod())

In [6]:
top = df[df['rank'] <= 5000].copy()
recent = top[top['date'] >= '2023-01-01'].copy()

In [24]:
recent['cumret'] = recent.groupby('ticker')['return'].transform(lambda x: (1 + x).cumprod())

In [7]:
recent['crosses'] = recent.groupby('ticker').close.transform(get_crossings)
recent['crosses5'] = recent.groupby('ticker').close.transform(lambda x: get_crossings(x, ma=5))
recent['crosses15'] = recent.groupby('ticker').close.transform(lambda x: get_crossings(x, ma=15))
recent['crosses30'] = recent.groupby('ticker').close.transform(lambda x: get_crossings(x, ma=30))

In [25]:
between = get_between(recent)

In [26]:
between['std'] = between.groupby('ticker')['return'].transform(lambda x: x.std())
between['stddev'] = between['std'].transform(normalize)
between['crossnorm'] = between['crosses'].transform(normalize)


In [42]:
between['score'] = np.sqrt(0.1*between['stddev']**2 + between['crossnorm']**2)

In [43]:
between.groupby('ticker')['score'].mean().nlargest(10)

ticker
VERB     9.508580
MNTS     9.380462
BOIL     5.420929
SPRC     4.695778
KNG      3.632566
HLN      3.632112
DGICA    3.526119
ELS      3.526094
CLOV     3.422177
BNDX     3.314917
Name: score, dtype: float64

In [ ]:
# RZC, HLN, KNG, DGICA, BNDX, CHSCL, CHSCM, HON, GNR, AB, SLCA